In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import wfdb
from wfdb import processing

In [ ]:
# Demo 1 - Read a wfdb record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data.
record = wfdb.rdrecord('./data/418')
# rdrecordを使ってwfdbレコードを読む
wfdb.plot_wfdb(record = record, title = "Record 418 from vfdb")
#プロット

In [ ]:
display(record.__dict__)
# メタデータを見る

In [ ]:
signals, fields = wfdb.rdsamp('./data/418', sampfrom=100, sampto=15000)

In [ ]:
display(signals)

In [ ]:
display(fields)

In [ ]:
signals, fields = wfdb.rdsamp('./data/418', sampfrom=100, sampto=15000)

In [ ]:
display(signals)

In [ ]:
# Demo 4 - Read part of a WFDB annotation file into a wfdb.Annotation object, and plot the samples
annotation = wfdb.rdann('./data/418', 'atr',sampfrom=300000, sampto =520000)

In [ ]:
wfdb.plot_wfdb(annotation=annotation, time_units='seconds')

In [ ]:
# Demo 5 - Read a WFDB record and annotation. Plot all channels, and the annotation on top of channel 0.
record = wfdb.rdrecord('./data/418')

In [ ]:
annotation = wfdb.rdann('./data/418', 'atr')

In [ ]:
wfdb.plot_wfdb(record = record, annotation = annotation,  time_units = 'minutes')

In [ ]:
# Demo 19 - Use the gqrs detection algorithm and correct the peaks

def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()

In [ ]:
# Load the wfdb record and the physical samples
record = wfdb.rdrecord('./data/418', sampfrom=0, sampto=10000, channels=[0])

In [ ]:
# Use the gqrs algorithm to detect qrs locations in the first channel
qrs_inds = processing.gqrs_detect(sig=record.p_signal[:,0], fs=record.fs)

In [ ]:
# Plot results
peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
        title="GQRS peak detection on record 418")

In [ ]:
# Correct the peaks shifting them to local maxima
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.correct_peaks(record.p_signal[:,0], peak_inds=qrs_inds,
                                               search_radius=search_radius, smooth_window_size=150)


In [ ]:
# Display results
print('Corrected gqrs detected peak indices:', sorted(corrected_peak_inds))
peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/418")

In [ ]:
sig, fields = wfdb.rdsamp('data/418', channels=[0],sampfrom = 100000, sampto=300000)
ann_ref = wfdb.rdann('data/418','atr', sampfrom = 100000,sampto=300000)

In [ ]:
# Run qrs detection on signal
xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
xqrs.detect()

In [ ]:
# Compare detected qrs complexes to reference annotation.
# Note, first sample in 100.atr is not a qrs.
comparitor = processing.compare_annotations(ref_sample=ann_ref.sample[1:],
                                            test_sample=xqrs.qrs_inds,
                                            window_width=int(0.1 * fields['fs']),
                                            signal=sig[:,0])

In [ ]:
# Print and plot the results
comparitor.print_summary()
comparitor.plot(title='xqrs detected qrs vs reference annotations')